# Spark training from sparkbyexamples.com

## PySpark Join Types

### Setup environment

In [38]:
import os
import sys
import pyspark

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pyspark
import pyspark.pandas as ps
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType
from pyspark.sql.functions import col, struct, when, lit, expr, sum, avg, max, min, mean, count 
from pyspark.sql.functions import udf, upper, transform, explode, map_keys, map_values
from pyspark.sql import Row

from pyspark.sql.functions import approx_count_distinct, avg, collect_list, collect_set, countDistinct, count, grouping, first, last
from pyspark.sql.functions import kurtosis, max, min, mean, skewness, stddev, stddev_samp, stddev_pop, sum, sumDistinct
from pyspark.sql.functions import variance, var_samp, var_pop, sum_distinct

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, percent_rank, ntile, cume_dist, lag, lead, col,avg,sum,min,max

spark = SparkSession.builder.master('local[1]') \
    .appName('SparkByExamples.com') \
    .getOrCreate()

### Create base data frames

In [2]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



### Inner join

In [3]:
empDF.join(deptDF,empDF.emp_dept_id == deptDF.dept_id, 'inner').show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### Full outer join

In [4]:
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'outer').show(truncate=False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'full').show(truncate=False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'fullouter').show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|null  |null    |null           |null       |null       |null  |null  |Sales    |30     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |null     |null   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+-

### Left outer join

In [5]:
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'left').show(truncate=False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'leftouter').show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|6     |Brown   |2              |2010       |50         |      |-1    |null     |null   |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|n

### Right outer join

In [6]:
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'right').show(truncate=False)
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'rightouter').show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|null  |null    |null           |null       |null       |null  |null  |Sales    |30     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|n

 ### Left semi join

In [7]:
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'leftsemi').show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+



### Left anti join

In [8]:
empDF.join(deptDF, empDF.emp_dept_id == deptDF.dept_id, 'leftanti').show(truncate=False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+



### Self join

In [11]:
empDF.alias('emp1').join(empDF.alias('emp2'), \
        col('emp1.superior_emp_id') == col('emp2.emp_id'), 'inner') \
        .select(col('emp1.emp_id'), col('emp1.name'), \
               col('emp2.emp_id').alias('superior_emp_id'), \
               col('emp2.name').alias('superior_emp_name')) \
        .show(truncate=False)

+------+--------+---------------+-----------------+
|emp_id|name    |superior_emp_id|superior_emp_name|
+------+--------+---------------+-----------------+
|2     |Rose    |1              |Smith            |
|3     |Williams|1              |Smith            |
|4     |Jones   |2              |Rose             |
|5     |Brown   |2              |Rose             |
|6     |Brown   |2              |Rose             |
+------+--------+---------------+-----------------+



### Using sql expression

In [12]:
empDF.createOrReplaceTempView('EMP')
deptDF.createOrReplaceTempView('DEPT')

joinDF = spark.sql('select * from EMP e, DEPT d where e.emp_dept_id == d.dept_id') \
    .show(truncate=False)

joinDF2 = spark.sql('select * from EMP e inner join DEPT d on e.emp_dept_id == d.dept_id') \
    .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-

## PySpark union and union all

### Create data frame

In [15]:
simpleData = [("James","Sales","NY",90000,34,10000), \
    ("Michael","Sales","NY",86000,56,20000), \
    ("Robert","Sales","CA",81000,30,23000), \
    ("Maria","Finance","CA",90000,24,23000) \
  ]

columns= ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

simpleData2 = [("James","Sales","NY",90000,34,10000), \
    ("Maria","Finance","CA",90000,24,23000), \
    ("Jen","Finance","NY",79000,53,15000), \
    ("Jeff","Marketing","CA",80000,25,18000), \
    ("Kumar","Marketing","NY",91000,50,21000) \
  ]
columns2= ["employee_name","department","state","salary","age","bonus"]

df2 = spark.createDataFrame(data = simpleData2, schema = columns2)

df2.printSchema()
df2.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
+-------------+----------+-----+------+---+-----+

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- state: string (nullable = true)
 |-- salary: long (nullable = true)
 |-- age: long (nullable = true)
 |-- bonus: long (nullable = true)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----

### Merge two dataframes with union

In [18]:
unionDF = df.union(df2)
unionDF.show(truncate=False)
print('Count of merged dataframes: ' + str(unionDF.count()))

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|James        |Sales     |NY   |90000 |34 |10000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+

Count of merged dataframes: 9


### Merge two dataframes using union all
Union All has been deprecated since PySpark 2.0.0.  It is recommended that you use union instead.

In [19]:
unionAllDF = df.unionAll(df2)
unionAllDF.show(truncate=False)
print('Count of merged dataframes: ' + str(unionAllDF.count()))

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|James        |Sales     |NY   |90000 |34 |10000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+

Count of merged dataframes: 9


### Merge without duplicates

In [20]:
disDF = df.union(df2).distinct()
disDF.show(truncate=False)
print('Count of merged dataframes: ' + str(disDF.count()))

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
+-------------+----------+-----+------+---+-----+

Count of merged dataframes: 7


## PySpark unionByName() 
### Build base data frames

In [21]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# Create DataFrame df1 with columns name, and id
data = [("James",34), ("Michael",56), \
        ("Robert",30), ("Maria",24) ]

df1 = spark.createDataFrame(data = data, schema=["name","id"])
df1.printSchema()

# Create DataFrame df2 with columns name and id
data2=[(34,"James"),(45,"Maria"), \
       (45,"Jen"),(34,"Jeff")]

df2 = spark.createDataFrame(data = data2, schema = ["id","name"])
df2.printSchema()

root
 |-- name: string (nullable = true)
 |-- id: long (nullable = true)

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)



### unionByName() example

In [22]:
df3 = df1.unionByName(df2)
df3.printSchema()
df3.show(truncate=False)
print('Count of rows in new dataframe: ' + str(df3.count()))

root
 |-- name: string (nullable = true)
 |-- id: long (nullable = true)

+-------+---+
|name   |id |
+-------+---+
|James  |34 |
|Michael|56 |
|Robert |30 |
|Maria  |24 |
|James  |34 |
|Maria  |45 |
|Jen    |45 |
|Jeff   |34 |
+-------+---+

Count of rows in new dataframe: 8


### unionByName with different columns

In [23]:
# Create DataFrames with different column names
df1 = spark.createDataFrame([[5, 2, 6]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[6, 7, 3]], ["col1", "col2", "col3"])

# Using allowMissingColumns
df3 = df1.unionByName(df2, allowMissingColumns=True)
df3.printSchema
df3.show()

+----+----+----+----+
|col0|col1|col2|col3|
+----+----+----+----+
|   5|   2|   6|null|
|null|   6|   7|   3|
+----+----+----+----+



## PySpark User Defined Function
### Build base dataframes

In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



### Create user defined function

In [34]:
def convertCase(str):
    resStr = ''
    arr = str.split(' ')
    for x in arr:
        resStr = resStr + x[0:1].upper() + x[1:len(x)] + ' '
        
    return resStr

### Convert python function to UDF

In [35]:
convertUDF = udf(lambda z: convertCase(z), StringType())

### Use UDF with dataframe

In [36]:
print(convertCase('here'))

df.select(col('Seqno'), \
    convertUDF(col('Name')).alias('Name') ) \
    .show(truncate=False)

Here 
+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+



### Using UDF with PySpark dataframe withColumn()

In [38]:
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z: upperCase(z), StringType())

df.withColumn('Cureated Name', upperCaseUDF(col('Name'))).show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



### Registering PySpark UDF and use it on SQL

In [41]:
spark.udf.register('convertUDF', convertCase, StringType())
df.createOrReplaceTempView('NAME_TABLE')
spark.sql('select Seqno, convertUDF(Name) as Name from NAME_TABLE') \
    .show(truncate=False)

+-----+-------------+
|Seqno|Name         |
+-----+-------------+
|1    |John Jones   |
|2    |Tracey Smith |
|3    |Amy Sanders  |
+-----+-------------+



23/04/26 20:57:53 WARN SimpleFunctionRegistry: The function convertudf replaced a previously registered function.


### Creating UDF using annotation

In [42]:
@udf(returnType=StringType())
def upperCase(str):
    return str.upper()

df.withColumn('Cureated Name', upperCase(col('Name'))).show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



### Handling null values

In [43]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")

spark.sql("select convertUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
|4    |null        |
+-----+------------+



23/04/26 21:02:54 ERROR Executor: Exception in task 0.0 in stage 122.0 (TID 92)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/var/folders/k4/wb623tjn7rscpqqh2tbnc62r0000gn/T/ipykernel_1450/1659673845.py", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:514)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Itera

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/var/folders/k4/wb623tjn7rscpqqh2tbnc62r0000gn/T/ipykernel_1450/1659673845.py", line 3, in convertCase
AttributeError: 'NoneType' object has no attribute 'split'


In [44]:
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)   

+------------------+
|_nullsafeUDF(Name)|
+------------------+
|John Jones        |
|Tracey Smith      |
|Amy Sanders       |
|                  |
+------------------+

+-----+-----------+
|Seqno|Name       |
+-----+-----------+
|1    |John Jones |
+-----+-----------+



## PySpark transform() function

### Create base data

In [5]:
simpleData = (('Java',4000,5), \
             ('Python',4600,10), \
             ('Scala',4500,15), \
              ('Scala',4100,15), \
             ('PHP',3000,20), \
             )

columns = ['CourseName','fee','discount']

#Create data frame
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)


root
 |-- CourseName: string (nullable = true)
 |-- fee: long (nullable = true)
 |-- discount: long (nullable = true)

+----------+----+--------+
|CourseName|fee |discount|
+----------+----+--------+
|Java      |4000|5       |
|Python    |4600|10      |
|Scala     |4500|15      |
|Scala     |4100|15      |
|PHP       |3000|20      |
+----------+----+--------+



### Create custom functions/transformation

In [64]:
#Create first custom transformation
def to_upper_str_columns(df):
    return df.withColumn('CourseName',upper(df.CourseName))

#Create second custom transformation
def reduce_price(reduceBy):
    def inner(df):
        return df.withColumn('new_fee', df.fee - reduceBy)
    return inner

#Create third custom transformation
def apply_discount(df):
    return df.withColumn('discounted_fee', \
                        df.new_fee - (df.new_fee * df.discount) / 100)

#Create fourth transformation
def select_columns(df):
    return df.select('CourseName','discounted_fee')


### PySpark apply dataframe transformations

In [65]:
#df2 = df.transform(to_upper_str_columns) \
        #.transform(reduce_price,1000) \
#.transform(apply_discount)
#df2 = df.transform(reduce_price,1000)
df2 = df.transform(to_upper_str_columns) \
    .transform(reduce_price(1000)) \
    .transform(apply_discount) 
df2.printSchema()
df2.show(truncate=False)

df2 = df.transform(to_upper_str_columns) \
    .transform(reduce_price(1000)) \
    .transform(apply_discount) \
    .transform(select_columns)
df2.printSchema()
df2.show(truncate=False)




root
 |-- CourseName: string (nullable = true)
 |-- fee: long (nullable = true)
 |-- discount: long (nullable = true)
 |-- new_fee: long (nullable = true)
 |-- discounted_fee: double (nullable = true)

+----------+----+--------+-------+--------------+
|CourseName|fee |discount|new_fee|discounted_fee|
+----------+----+--------+-------+--------------+
|JAVA      |4000|5       |3000   |2850.0        |
|PYTHON    |4600|10      |3600   |3240.0        |
|SCALA     |4500|15      |3500   |2975.0        |
|SCALA     |4100|15      |3100   |2635.0        |
|PHP       |3000|20      |2000   |1600.0        |
+----------+----+--------+-------+--------------+

root
 |-- CourseName: string (nullable = true)
 |-- discounted_fee: double (nullable = true)

+----------+--------------+
|CourseName|discounted_fee|
+----------+--------------+
|JAVA      |2850.0        |
|PYTHON    |3240.0        |
|SCALA     |2975.0        |
|SCALA     |2635.0        |
|PHP       |1600.0        |
+----------+--------------+



## PySpark sql.functions.transform

### Create base data

In [66]:
data = [
 ("James,,Smith",["Java","Scala","C++"],["Spark","Java"]),
 ("Michael,Rose,",["Spark","Java","C++"],["Spark","Java"]),
 ("Robert,,Williams",["CSharp","VB"],["Spark","Python"])
]
df = spark.createDataFrame(data=data,schema=["Name","Languages1","Languages2"])
df.printSchema()
df.show()

root
 |-- Name: string (nullable = true)
 |-- Languages1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Languages2: array (nullable = true)
 |    |-- element: string (containsNull = true)

+----------------+------------------+---------------+
|            Name|        Languages1|     Languages2|
+----------------+------------------+---------------+
|    James,,Smith|[Java, Scala, C++]|  [Spark, Java]|
|   Michael,Rose,|[Spark, Java, C++]|  [Spark, Java]|
|Robert,,Williams|      [CSharp, VB]|[Spark, Python]|
+----------------+------------------+---------------+



### Use transform() function

In [67]:
df.select(transform('Languages1', lambda x: upper(x)).alias('languages1')).show(truncate=False)

+------------------+
|languages1        |
+------------------+
|[JAVA, SCALA, C++]|
|[SPARK, JAVA, C++]|
|[CSHARP, VB]      |
+------------------+



## PySpark appl function to column

Create base data

In [68]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+



### apply function using withColumn()

In [69]:
df.withColumn('Upper_Name', upper(df.Name)).show(truncate=False)

+-----+------------+------------+
|Seqno|Name        |Upper_Name  |
+-----+------------+------------+
|1    |john jones  |JOHN JONES  |
|2    |tracey smith|TRACEY SMITH|
|3    |amy sanders |AMY SANDERS |
+-----+------------+------------+



### apply function using select

In [72]:
df.select('Seqno','Name',upper(df.Name).alias('Upper_Name')).show()

+-----+------------+------------+
|Seqno|        Name|  Upper_Name|
+-----+------------+------------+
|    1|  john jones|  JOHN JONES|
|    2|tracey smith|TRACEY SMITH|
|    3| amy sanders| AMY SANDERS|
+-----+------------+------------+



In [73]:
df.createOrReplaceTempView('TAB')
spark.sql('select Seqno, Name, upper(Name) as Upper_Name from TAB').show()

+-----+------------+------------+
|Seqno|        Name|  Upper_Name|
+-----+------------+------------+
|    1|  john jones|  JOHN JONES|
|    2|tracey smith|TRACEY SMITH|
|    3| amy sanders| AMY SANDERS|
+-----+------------+------------+



### Create custom function

In [74]:
def upperCase(str):
    return str.upper()

### Register UDF

In [75]:
upperCaseUDF = udf(lambda x:upperCase(x), StringType())

### Apply custom UDF to column

In [76]:
# Custom UDF using withColumn
df.withColumn('Cureated Name', upperCaseUDF(col('Name'))).show(truncate=False)

# Custom UDF with select()
df.select(col('Seqno'), \
         upperCaseUDF(col('Name')).alias('Name')).show(truncate=False)

# Custom UDF with sql
spark.udf.register('upperCaseUDF', upperCaseUDF)
df.createOrReplaceTempView('TAB')
spark.sql('select Seqno, Name, upperCaseUDF(Name) as Upper_Name from TAB').show(truncate=False)

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |JOHN JONES  |
|2    |TRACEY SMITH|
|3    |AMY SANDERS |
+-----+------------+

+-----+------------+------------+
|Seqno|Name        |Upper_Name  |
+-----+------------+------------+
|1    |john jones  |JOHN JONES  |
|2    |tracey smith|TRACEY SMITH|
|3    |amy sanders |AMY SANDERS |
+-----+------------+------------+



### PySpark Pandas apply()

In [78]:
technologies = ({
    'Fee' :[20000,25000,30000,22000,np.NaN],
    'Discount':[1000,2500,1500,1200,3000]
               })

# Create data frame
psdf = ps.DataFrame(technologies)
print(psdf)

def add(data):
    return data[0] + data[1]

addDF = psdf.apply(add, axis=1)
print(addDF)

C:\anaconda3\lib\site-packages\pyspark\pandas\internal.py:1536: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [


       Fee  Discount
0  20000.0      1000
1  25000.0      2500
2  30000.0      1500
3  22000.0      1200
4      NaN      3000


C:\anaconda3\lib\site-packages\pyspark\pandas\internal.py:1536: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [


0    21000.0
1    27500.0
2    31500.0
3    23200.0
4        NaN
dtype: float64


## PySpark map() transformations

Create base data

In [79]:
data = ["Project","Gutenberg’s","Alice’s","Adventures",
"in","Wonderland","Project","Gutenberg’s","Adventures",
"in","Wonderland","Project","Gutenberg’s"]

rdd=spark.sparkContext.parallelize(data)

### PySpark map() example with RDD

In [80]:
rdd2 = rdd.map(lambda x: (x,1))
for element in rdd2.collect():
    print(element)

('Project', 1)
('Gutenberg’s', 1)
('Alice’s', 1)
('Adventures', 1)
('in', 1)
('Wonderland', 1)
('Project', 1)
('Gutenberg’s', 1)
('Adventures', 1)
('in', 1)
('Wonderland', 1)
('Project', 1)
('Gutenberg’s', 1)


### PySpark map() example with data frame

In [81]:
data = [('James','Smith','M',30),
       ('Anna','Rose','F',41),
       ('Robert','Williams','M',62),
       ]

columns = ['firstname','lastname','gender','salary']

df = spark.createDataFrame(data=data, schema=columns)
df.show()

+---------+--------+------+------+
|firstname|lastname|gender|salary|
+---------+--------+------+------+
|    James|   Smith|     M|    30|
|     Anna|    Rose|     F|    41|
|   Robert|Williams|     M|    62|
+---------+--------+------+------+



In [84]:
# Refering to columns by index
rdd2 = df.rdd.map(lambda x: (x[1] + ', '+ x[0],x[2],x[3]*2))
df2=rdd2.toDF(['name','gender','new_salary'])
df2.show()

+----------------+------+----------+
|            name|gender|new_salary|
+----------------+------+----------+
|    Smith, James|     M|        60|
|      Rose, Anna|     F|        82|
|Williams, Robert|     M|       124|
+----------------+------+----------+



In [85]:
# Refering to column names
rdd2=df.rdd.map(lambda x: (x['lastname'] + ', ' + x['firstname'], x['gender'], x['salary'] * 2))
df3 = rdd2.toDF(['name','gender','salary'])
df3.show()

+----------------+------+------+
|            name|gender|salary|
+----------------+------+------+
|    Smith, James|     M|    60|
|      Rose, Anna|     F|    82|
|Williams, Robert|     M|   124|
+----------------+------+------+



In [86]:
# Also refering to column names
rdd2 = df.rdd.map(lambda x: (x.lastname+', '+x.firstname, x.gender, x.salary*2))
df4 = rdd2.toDF(['name','gender','salary'])
df4.show()

+----------------+------+------+
|            name|gender|salary|
+----------------+------+------+
|    Smith, James|     M|    60|
|      Rose, Anna|     F|    82|
|Williams, Robert|     M|   124|
+----------------+------+------+



In [87]:
# By calling function
def func1(x):
    firstName=x.firstname
    lastName=x.lastname
    name=lastName + ', ' + firstName
    gender=x.gender.lower()
    salary=x.salary * 2
    
    return(name, gender, salary)

rdd2 = df.rdd.map(lambda x: func1(x))
df5 = rdd2.toDF(['name','gender','salary'])
df5.show()

+----------------+------+------+
|            name|gender|salary|
+----------------+------+------+
|    Smith, James|     m|    60|
|      Rose, Anna|     f|    82|
|Williams, Robert|     m|   124|
+----------------+------+------+



## PySpark flatMap() transformation

### Create base data

In [4]:
data = ["Project Gutenberg’s",
        "Alice’s Adventures in Wonderland",
        "Project Gutenberg’s",
        "Adventures in Wonderland",
        "Project Gutenberg’s"]
rdd=spark.sparkContext.parallelize(data)
for element in rdd.collect():
    print(element)

Project Gutenberg’s
Alice’s Adventures in Wonderland
Project Gutenberg’s
Adventures in Wonderland
Project Gutenberg’s


In [5]:
rdd2=rdd.flatMap(lambda x: x.split(' '))
for element in rdd2.collect():
    print(element)

Project
Gutenberg’s
Alice’s
Adventures
in
Wonderland
Project
Gutenberg’s
Adventures
in
Wonderland
Project
Gutenberg’s


### Using flatMap() transformation on dataframe

In [10]:
arrayData = [
        ('James',['Java','Scala'],{'hair':'black','eye':'brown'}),
        ('Michael',['Spark','Java',None],{'hair':'brown','eye':None}),
        ('Robert',['CSharp',''],{'hair':'red','eye':''}),
        ('Washington',None,None),
        ('Jefferson',['1','2'],{}), 
        ('Ron',['Python','Scala','SQL','VB'],{'hair':'black','eye':'brown'})]
df = spark.createDataFrame(data=arrayData, schema = ['name','knownLanguages','properties'])

df2 = df.select(df.name, explode(df.knownLanguages))
df2.printSchema()
df2.show(truncate=False)


root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)

+---------+------+
|name     |col   |
+---------+------+
|James    |Java  |
|James    |Scala |
|Michael  |Spark |
|Michael  |Java  |
|Michael  |null  |
|Robert   |CSharp|
|Robert   |      |
|Jefferson|1     |
|Jefferson|2     |
|Ron      |Python|
|Ron      |Scala |
|Ron      |SQL   |
|Ron      |VB    |
+---------+------+



## PySpark DataFrame foreach()

### Create base data

In [12]:
#create data
columns = ['Seqno','Name']

data = [('1','john jones'),
       ('2','tracey smith'),
       ('3','amy sanders')]

#create data frame
df = spark.createDataFrame(data=data,schema=columns)
df.show()

#foreach example
def f(df):
    print(df.Seqno)
    
df.foreach(f)


+-----+------------+
|Seqno|        Name|
+-----+------------+
|    1|  john jones|
|    2|tracey smith|
|    3| amy sanders|
+-----+------------+



1
2
3


### foreach with accumulator example

In [13]:
accum = spark.sparkContext.accumulator(0)
df.foreach(lambda x:accum.add(int(x.Seqno)))
print(accum.value)

6


### PySpark RDD foreach() usage

In [14]:
accum = spark.sparkContext.accumulator(0)
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
rdd.foreach(lambda x:accum.add(x))
print(accum.value)

15


## PySpark Random Sample

### Using fraction to get a random sample

In [3]:
# Create data frame
df = spark.range(100)
print(df.sample(0.06).collect())

[Row(id=3), Row(id=20), Row(id=25), Row(id=54), Row(id=65), Row(id=76), Row(id=84), Row(id=98)]


### Using seed to reproduce the same samples

In [4]:
print(df.sample(0.1,123).collect())

print(df.sample(0.1,123).collect())

print(df.sample(0.1,456).collect())

[Row(id=36), Row(id=37), Row(id=41), Row(id=43), Row(id=56), Row(id=66), Row(id=69), Row(id=75), Row(id=83)]
[Row(id=36), Row(id=37), Row(id=41), Row(id=43), Row(id=56), Row(id=66), Row(id=69), Row(id=75), Row(id=83)]
[Row(id=19), Row(id=21), Row(id=42), Row(id=48), Row(id=49), Row(id=50), Row(id=75), Row(id=80)]


### Stratified sampling

In [5]:
df2 = df.select((df.id % 3).alias('key'))
print(df2.sampleBy('key', {0: 0.1, 1: 0.2}).collect())

[Row(key=1), Row(key=1), Row(key=0), Row(key=0), Row(key=1), Row(key=0), Row(key=1), Row(key=1), Row(key=0), Row(key=0), Row(key=1)]


## PySpark fillna() and fill() - Replace null/none values

In [3]:
filePath = 'resources/small_zipcode.csv'
df = spark.read.options(header='true', inferSchema='true').csv(filePath)
df.printSchema()
df.show(truncate=False)

root
 |-- id: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- population: integer (nullable = true)

+---+-------+--------+-------------------+-----+----------+
|id |zipcode|type    |city               |state|population|
+---+-------+--------+-------------------+-----+----------+
|1  |704    |STANDARD|null               |PR   |30100     |
|2  |704    |null    |PASEO COSTA DEL SUR|PR   |null      |
|3  |709    |null    |BDA SAN LUIS       |PR   |3700      |
|4  |76166  |UNIQUE  |CINGULAR WIRELESS  |TX   |84000     |
|5  |76177  |STANDARD|null               |TX   |null      |
+---+-------+--------+-------------------+-----+----------+



### PySpark replace null/none values with zero

In [4]:
#Replace 0 for null for all integer columns
df.na.fill(value=0).show()
#Replace 0 for null on only population column
df.na.fill(value=0, subset=['population']).show()

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               null|   TX|         0|
+---+-------+--------+-------------------+-----+----------+

+---+-------+--------+-------------------+-----+----------+
| id|zipcode|    type|               city|state|population|
+---+-------+--------+-------------------+-----+----------+
|  1|    704|STANDARD|               null|   PR|     30100|
|  2|    704|    null|PASEO COSTA DEL SUR|   PR|         0|
|  3|    709|    null|       BDA SAN LUIS|   PR|      3700|
|  4|  76166|  UNIQUE|  CINGULAR WIRELESS|   TX|     84000|
|  5|  76177|STANDARD|               nu

### PySpark replace null/none with empty string

In [7]:
df.na.fill('').show(truncate=False)

+---+-------+--------+-------------------+-----+----------+
|id |zipcode|type    |city               |state|population|
+---+-------+--------+-------------------+-----+----------+
|1  |704    |STANDARD|                   |PR   |30100     |
|2  |704    |        |PASEO COSTA DEL SUR|PR   |null      |
|3  |709    |        |BDA SAN LUIS       |PR   |3700      |
|4  |76166  |UNIQUE  |CINGULAR WIRELESS  |TX   |84000     |
|5  |76177  |STANDARD|                   |TX   |null      |
+---+-------+--------+-------------------+-----+----------+



In [8]:
df.na.fill('unknown', ['city']).na.fill('',['type']).show(truncate=False)

+---+-------+--------+-------------------+-----+----------+
|id |zipcode|type    |city               |state|population|
+---+-------+--------+-------------------+-----+----------+
|1  |704    |STANDARD|unknown            |PR   |30100     |
|2  |704    |        |PASEO COSTA DEL SUR|PR   |null      |
|3  |709    |        |BDA SAN LUIS       |PR   |3700      |
|4  |76166  |UNIQUE  |CINGULAR WIRELESS  |TX   |84000     |
|5  |76177  |STANDARD|unknown            |TX   |null      |
+---+-------+--------+-------------------+-----+----------+



In [12]:
df.na.fill({'city':'unknown', 'type':''}).show(truncate=False)

+---+-------+--------+-------------------+-----+----------+
|id |zipcode|type    |city               |state|population|
+---+-------+--------+-------------------+-----+----------+
|1  |704    |STANDARD|unknown            |PR   |30100     |
|2  |704    |        |PASEO COSTA DEL SUR|PR   |null      |
|3  |709    |        |BDA SAN LUIS       |PR   |3700      |
|4  |76166  |UNIQUE  |CINGULAR WIRELESS  |TX   |84000     |
|5  |76177  |STANDARD|unknown            |TX   |null      |
+---+-------+--------+-------------------+-----+----------+



## PySpark Pivot and Unpivot DataFrame

In [13]:
data = [("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"), \
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"), \
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"), \
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico")]

columns= ["Product","Amount","Country"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)



+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



### Pivot PySpark DataFrame

In [14]:
pivotDF = df.groupby('Product').pivot('Country').sum('Amount')
pivotDF.printSchema()
pivotDF.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Canada: long (nullable = true)
 |-- China: long (nullable = true)
 |-- Mexico: long (nullable = true)
 |-- USA: long (nullable = true)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |2000  |1600|
|Banana |2000  |400  |null  |1000|
|Carrots|2000  |1200 |null  |1500|
+-------+------+-----+------+----+



In [15]:
countries = ['USA','China','Canada','Mexico']
pivotDF = df.groupby('Product').pivot('Country', countries).sum('Amount')
df.printSchema()
df.show(truncate=False)

root
 |-- Product: string (nullable = true)
 |-- Amount: long (nullable = true)
 |-- Country: string (nullable = true)

+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
|Banana |1000  |USA    |
|Carrots|1500  |USA    |
|Beans  |1600  |USA    |
|Orange |2000  |USA    |
|Orange |2000  |USA    |
|Banana |400   |China  |
|Carrots|1200  |China  |
|Beans  |1500  |China  |
|Orange |4000  |China  |
|Banana |2000  |Canada |
|Carrots|2000  |Canada |
|Beans  |2000  |Mexico |
+-------+------+-------+



In [16]:
pivotDF = df.groupby('Product','Country') \
    .sum('Amount') \
    .groupby('Product') \
    .pivot('Country') \
    .sum('sum(Amount)') 

pivotDF.show(truncate=False)

+-------+------+-----+------+----+
|Product|Canada|China|Mexico|USA |
+-------+------+-----+------+----+
|Orange |null  |4000 |null  |4000|
|Beans  |null  |1500 |2000  |1600|
|Banana |2000  |400  |null  |1000|
|Carrots|2000  |1200 |null  |1500|
+-------+------+-----+------+----+



### Unpivot PySpark DataFrame

In [17]:
unpivotExpr = "stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"
unPivotDF = pivotDF.select("Product", expr(unpivotExpr)) \
    .where("Total is not null")
unPivotDF.show(truncate=False)
unPivotDF.show()

+-------+-------+-----+
|Product|Country|Total|
+-------+-------+-----+
|Orange |China  |4000 |
|Beans  |China  |1500 |
|Beans  |Mexico |2000 |
|Banana |Canada |2000 |
|Banana |China  |400  |
|Carrots|Canada |2000 |
|Carrots|China  |1200 |
+-------+-------+-----+

+-------+-------+-----+
|Product|Country|Total|
+-------+-------+-----+
| Orange|  China| 4000|
|  Beans|  China| 1500|
|  Beans| Mexico| 2000|
| Banana| Canada| 2000|
| Banana|  China|  400|
|Carrots| Canada| 2000|
|Carrots|  China| 1200|
+-------+-------+-----+



## PySpark partitionBy() - Write to disk example

In [18]:
df = spark.read.option('header',True).csv('resources/simple-zipcodes.csv')
df.printSchema()
df.show(truncate=False)

root
 |-- RecordNumber: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- State: string (nullable = true)

+------------+-------+-------------------+-------+-----+
|RecordNumber|Country|City               |Zipcode|State|
+------------+-------+-------------------+-------+-----+
|1           |US     |PARC PARQUE        |704    |PR   |
|2           |US     |PASEO COSTA DEL SUR|704    |PR   |
|10          |US     |BDA SAN LUIS       |709    |PR   |
|49347       |US     |HOLT               |32564  |FL   |
|49348       |US     |HOMOSASSA          |34487  |FL   |
|61391       |US     |CINGULAR WIRELESS  |76166  |TX   |
|61392       |US     |FORT WORTH         |76177  |TX   |
|61393       |US     |FT WORTH           |76177  |TX   |
|54356       |US     |SPRUCE PINE        |35585  |AL   |
|76511       |US     |ASH HILL           |27007  |NC   |
|4           |US     |URB EUGENE RICE    |704    |PR   |
|

### PySpark partitionBy()

In [19]:
df.write.option('header',True) \
    .partitionBy('state') \
    .mode('overwrite') \
    .csv('resource/zipcodes-state')

### PySpark partitionBy() multiple columns

In [20]:
df.write.option('header',True) \
    .partitionBy('state','city') \
    .mode('overwrite') \
    .csv('resource/zipcodes-state')

### Using repartition() and partitionBy() together

In [23]:
df.repartition(2) \
    .write.option('header',True) \
    .partitionBy('state') \
    .mode('overwrite') \
    .csv('resource/zipcodes-state-more')

### Data Skew - Control number of records per file

In [24]:
df.write.option('header',True) \
  .option('maxRecordsPerFile',2) \
  .partitionBy('state') \
  .mode('overwrite') \
  .csv('resource/tmp/zipcodes-state')

### Read a specific partition

In [25]:
dfSinglePart = spark.read.option('header',True) \
  .csv('resource/zipcodes-state/state=AL/city=SPRINGVILLE')
dfSinglePart.printSchema()
dfSinglePart.show(truncate=False)

root
 |-- RecordNumber: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Zipcode: string (nullable = true)

+------------+-------+-------+
|RecordNumber|Country|Zipcode|
+------------+-------+-------+
|54355       |US     |35146  |
+------------+-------+-------+



### PySpark SQL - Read Partition Data

In [26]:
parqDF = spark.read.option('header',True) \
  .csv('resource/zipcodes-state')

parqDF.createOrReplaceTempView('ZIPCODE')
spark.sql("select * from ZIPCODE where state = 'AL' and city = 'SPRINGVILLE'").show(truncate=False)

+------------+-------+-------+-----+-----------+
|RecordNumber|Country|Zipcode|state|city       |
+------------+-------+-------+-----+-----------+
|54355       |US     |35146  |AL   |SPRINGVILLE|
+------------+-------+-------+-----+-----------+



## PySpark MapType (Dict) Usage with Examples

### Create PySpark Map Type

In [27]:
mapCol = MapType(StringType(), StringType(),False)
print(mapCol)

MapType(StringType(), StringType(), False)


### Create MapType from StructType

In [28]:
schema = StructType([
    StructField('name', StringType(), True),
    StructField('properties', MapType(StringType(), StringType()),True)
])

print(schema)

StructType([StructField('name', StringType(), True), StructField('properties', MapType(StringType(), StringType(), True), True)])


### Create DataFrame using StructType schema

In [29]:
dataDictionary = [
    ('James',{'hair':'black','eye':'brown'}),
    ('Michael', {'hair':'brown', 'eye':None}),
    ('Robert', {'hair':'red', 'eye':'black'}),
    ('Washington', {'hair':'grey', 'eye':'grey'}),
    ('Jefferson', {'hair':'brown', 'eye':''})
]

df = spark.createDataFrame(data=dataDictionary,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+----------+-----------------------------+
|name      |properties                   |
+----------+-----------------------------+
|James     |{eye -> brown, hair -> black}|
|Michael   |{eye -> null, hair -> brown} |
|Robert    |{eye -> black, hair -> red}  |
|Washington|{eye -> grey, hair -> grey}  |
|Jefferson |{eye -> , hair -> brown}     |
+----------+-----------------------------+



### Access PySpark MapType Elements

In [31]:
df3=df.rdd.map(lambda x: \
    (x.name,x.properties["hair"],x.properties["eye"])) \
    .toDF(["name","hair","eye"])
df3.printSchema()
df3.show()

root
 |-- name: string (nullable = true)
 |-- hair: string (nullable = true)
 |-- eye: string (nullable = true)

+----------+-----+-----+
|      name| hair|  eye|
+----------+-----+-----+
|     James|black|brown|
|   Michael|brown| null|
|    Robert|  red|black|
|Washington| grey| grey|
| Jefferson|brown|     |
+----------+-----+-----+



In [32]:
df.withColumn('hair', df.properties.getItem('hair')) \
  .withColumn('eye', df.properties.getItem('eye')) \
  .drop('properties') \
  .show(truncate=False)

df.withColumn('hair',df.properties['hair']) \
  .withColumn('eye',df.properties['eye']) \
  .drop('properties') \
  .show(truncate=False)


+----------+-----+-----+
|name      |hair |eye  |
+----------+-----+-----+
|James     |black|brown|
|Michael   |brown|null |
|Robert    |red  |black|
|Washington|grey |grey |
|Jefferson |brown|     |
+----------+-----+-----+

+----------+-----+-----+
|name      |hair |eye  |
+----------+-----+-----+
|James     |black|brown|
|Michael   |brown|null |
|Robert    |red  |black|
|Washington|grey |grey |
|Jefferson |brown|     |
+----------+-----+-----+



### Functions - explode()

In [33]:
df.select(df.name,explode(df.properties)).show(truncate=False)

+----------+----+-----+
|name      |key |value|
+----------+----+-----+
|James     |eye |brown|
|James     |hair|black|
|Michael   |eye |null |
|Michael   |hair|brown|
|Robert    |eye |black|
|Robert    |hair|red  |
|Washington|eye |grey |
|Washington|hair|grey |
|Jefferson |eye |     |
|Jefferson |hair|brown|
+----------+----+-----+



### Functions - map_keys() - Get All Map Keys

In [35]:
from pyspark.sql.functions import map_keys
df.select(df.name,map_keys(df.properties)).show(truncate=False)

+----------+--------------------+
|name      |map_keys(properties)|
+----------+--------------------+
|James     |[eye, hair]         |
|Michael   |[eye, hair]         |
|Robert    |[eye, hair]         |
|Washington|[eye, hair]         |
|Jefferson |[eye, hair]         |
+----------+--------------------+



### Functions - map_values() - Get All Map Values

In [36]:
from pyspark.sql.functions import map_values
df.select(df.name, map_values(df.properties)).show(truncate=False)

+----------+----------------------+
|name      |map_values(properties)|
+----------+----------------------+
|James     |[brown, black]        |
|Michael   |[null, brown]         |
|Robert    |[black, red]          |
|Washington|[grey, grey]          |
|Jefferson |[, brown]             |
+----------+----------------------+



# PySpark SQL Functions

## PySpark Aggregate Functions

### Create dataframe

In [3]:
simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
schema = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



### approx_count_distinct()

In [11]:
from pyspark.sql.functions import approx_count_distinct, avg, collect_list, collect_set, countDistinct, count, grouping, first, last
from pyspark.sql.functions import kurtosis, max, min, mean, skewness, stddev, stddev_samp, stddev_pop, sum, sumDistinct
from pyspark.sql.functions import variance, var_samp, var_pop

print("approx_count_distinct: " + \
      str(df.select(approx_count_distinct("salary")).collect()[0][0]))

approx_count_distinct: 6


### avg()

In [14]:
print('avg: ' + str(df.select(avg('salary')).collect()[0][0]))

avg: 3400.0


### collect_list()

In [15]:
df.select(collect_list('salary')).show(truncate=False)

+------------------------------------------------------------+
|collect_list(salary)                                        |
+------------------------------------------------------------+
|[3000, 4600, 4100, 3000, 3000, 3300, 3900, 3000, 2000, 4100]|
+------------------------------------------------------------+



### collect_set()

In [16]:
df.select(collect_set('salary')).show(truncate=False)

+------------------------------------+
|collect_set(salary)                 |
+------------------------------------+
|[2000, 3300, 4100, 3000, 4600, 3900]|
+------------------------------------+



### countDistinct()

In [19]:
df2 = df.select(countDistinct('department','salary'))
df2.show(truncate=False)
print('Distinct count of department & salary: ' + str(df2.collect()[0][0]))

+----------------------------------+
|count(DISTINCT department, salary)|
+----------------------------------+
|8                                 |
+----------------------------------+

Distinct count of department & salary: 8


### first()

In [20]:
df.select(first('salary')).show(truncate=False)

+-------------+
|first(salary)|
+-------------+
|3000         |
+-------------+



### last()

In [21]:
df.select(last('salary')).show(truncate=False)

+------------+
|last(salary)|
+------------+
|4100        |
+------------+



### kurtosis()

In [22]:
df.select(kurtosis('salary')).show(truncate=False)

+-------------------+
|kurtosis(salary)   |
+-------------------+
|-0.6467803030303032|
+-------------------+



### max()

In [23]:
df.select(max('salary')).show(truncate=False)

+-----------+
|max(salary)|
+-----------+
|4600       |
+-----------+



### min()

In [24]:
df.select(min('salary')).show(truncate=False)

+-----------+
|min(salary)|
+-----------+
|2000       |
+-----------+



### mean()

In [25]:
df.select(mean('salary')).show(truncate=False)

+-----------+
|avg(salary)|
+-----------+
|3400.0     |
+-----------+



### skewness()

In [26]:
df.select(skewness('salary')).show(truncate=False)

+--------------------+
|skewness(salary)    |
+--------------------+
|-0.12041791181069571|
+--------------------+



### stddev(), stddev_samp(), and stddev_pop

In [27]:
df.select(stddev('salary'), stddev_samp('salary'), stddev_pop('salary')).show(truncate=False)

+-------------------+-------------------+------------------+
|stddev_samp(salary)|stddev_samp(salary)|stddev_pop(salary)|
+-------------------+-------------------+------------------+
|765.9416862050705  |765.9416862050705  |726.636084983398  |
+-------------------+-------------------+------------------+



### sum(), and sum_distinct()

In [31]:
df.select(sum('salary'), sum_distinct('salary')).show(truncate=False)

+-----------+--------------------+
|sum(salary)|sum(DISTINCT salary)|
+-----------+--------------------+
|34000      |20900               |
+-----------+--------------------+



### variance(), var_samp(), and var_pop()

In [32]:
df.select(variance('salary'), var_samp('salary'), var_pop('salary')).show(truncate=False)

+-----------------+-----------------+---------------+
|var_samp(salary) |var_samp(salary) |var_pop(salary)|
+-----------------+-----------------+---------------+
|586666.6666666666|586666.6666666666|528000.0       |
+-----------------+-----------------+---------------+



## PySpark Window Functions

### Create data

In [34]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )
 
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



### row_number() function

In [35]:
windowSpec = Window.partitionBy('department').orderBy('salary')

df.withColumn('row_number', row_number().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|row_number|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |2         |
|Robert       |Sales     |4100  |3         |
|Saif         |Sales     |4100  |4         |
|Michael      |Sales     |4600  |5         |
+-------------+----------+------+----------+



### rank() function

In [37]:
df.withColumn('rank', rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|rank|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |1   |
|Scott        |Finance   |3300  |2   |
|Jen          |Finance   |3900  |3   |
|Kumar        |Marketing |2000  |1   |
|Jeff         |Marketing |3000  |2   |
|James        |Sales     |3000  |1   |
|James        |Sales     |3000  |1   |
|Robert       |Sales     |4100  |3   |
|Saif         |Sales     |4100  |3   |
|Michael      |Sales     |4600  |5   |
+-------------+----------+------+----+



### dense_rank() function

In [39]:
df.withColumn('dense_rank', dense_rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+----------+
|employee_name|department|salary|dense_rank|
+-------------+----------+------+----------+
|Maria        |Finance   |3000  |1         |
|Scott        |Finance   |3300  |2         |
|Jen          |Finance   |3900  |3         |
|Kumar        |Marketing |2000  |1         |
|Jeff         |Marketing |3000  |2         |
|James        |Sales     |3000  |1         |
|James        |Sales     |3000  |1         |
|Robert       |Sales     |4100  |2         |
|Saif         |Sales     |4100  |2         |
|Michael      |Sales     |4600  |3         |
+-------------+----------+------+----------+



### percent_rank() function

In [40]:
df.withColumn('percent_rank',percent_rank().over(windowSpec)).show(truncate=False)

+-------------+----------+------+------------+
|employee_name|department|salary|percent_rank|
+-------------+----------+------+------------+
|Maria        |Finance   |3000  |0.0         |
|Scott        |Finance   |3300  |0.5         |
|Jen          |Finance   |3900  |1.0         |
|Kumar        |Marketing |2000  |0.0         |
|Jeff         |Marketing |3000  |1.0         |
|James        |Sales     |3000  |0.0         |
|James        |Sales     |3000  |0.0         |
|Robert       |Sales     |4100  |0.5         |
|Saif         |Sales     |4100  |0.5         |
|Michael      |Sales     |4600  |1.0         |
+-------------+----------+------+------------+



### ntile() function

In [42]:
df.withColumn('ntile', ntile(2).over(windowSpec)).show(truncate=False)

+-------------+----------+------+-----+
|employee_name|department|salary|ntile|
+-------------+----------+------+-----+
|Maria        |Finance   |3000  |1    |
|Scott        |Finance   |3300  |1    |
|Jen          |Finance   |3900  |2    |
|Kumar        |Marketing |2000  |1    |
|Jeff         |Marketing |3000  |2    |
|James        |Sales     |3000  |1    |
|James        |Sales     |3000  |1    |
|Robert       |Sales     |4100  |1    |
|Saif         |Sales     |4100  |2    |
|Michael      |Sales     |4600  |2    |
+-------------+----------+------+-----+



### cume_dist() function

In [43]:
df.withColumn('cume_dist',cume_dist().over(windowSpec)).show(truncate=False)

+-------------+----------+------+------------------+
|employee_name|department|salary|cume_dist         |
+-------------+----------+------+------------------+
|Maria        |Finance   |3000  |0.3333333333333333|
|Scott        |Finance   |3300  |0.6666666666666666|
|Jen          |Finance   |3900  |1.0               |
|Kumar        |Marketing |2000  |0.5               |
|Jeff         |Marketing |3000  |1.0               |
|James        |Sales     |3000  |0.4               |
|James        |Sales     |3000  |0.4               |
|Robert       |Sales     |4100  |0.8               |
|Saif         |Sales     |4100  |0.8               |
|Michael      |Sales     |4600  |1.0               |
+-------------+----------+------+------------------+



### lag() function

In [44]:
df.withColumn('lag',lag('salary',2).over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|lag |
+-------------+----------+------+----+
|Maria        |Finance   |3000  |null|
|Scott        |Finance   |3300  |null|
|Jen          |Finance   |3900  |3000|
|Kumar        |Marketing |2000  |null|
|Jeff         |Marketing |3000  |null|
|James        |Sales     |3000  |null|
|James        |Sales     |3000  |null|
|Robert       |Sales     |4100  |3000|
|Saif         |Sales     |4100  |3000|
|Michael      |Sales     |4600  |4100|
+-------------+----------+------+----+



### lead() function

In [45]:
df.withColumn('lead',lead('salary',2).over(windowSpec)).show(truncate=False)

+-------------+----------+------+----+
|employee_name|department|salary|lead|
+-------------+----------+------+----+
|Maria        |Finance   |3000  |3900|
|Scott        |Finance   |3300  |null|
|Jen          |Finance   |3900  |null|
|Kumar        |Marketing |2000  |null|
|Jeff         |Marketing |3000  |null|
|James        |Sales     |3000  |4100|
|James        |Sales     |3000  |4100|
|Robert       |Sales     |4100  |4600|
|Saif         |Sales     |4100  |null|
|Michael      |Sales     |4600  |null|
+-------------+----------+------+----+

